In [0]:
label = {0:"letter",1:"form",2:"email",3:"handwritten",4:"advertisement",
         5:"scientific report",6:"scientific publication",7:"specification",
         8:"file folder",9:"news article",10:"budget",11:"invoice",12:"presentation",
         13:"questionnaire",14:"resume",15:"memo"}

In [5]:
!wget https://aip.scitation.org/pb-assets/images/sample_supplementary_material-1549471999507.jpg

--2019-11-20 06:58:30--  https://aip.scitation.org/pb-assets/images/sample_supplementary_material-1549471999507.jpg
Resolving aip.scitation.org (aip.scitation.org)... 104.17.163.62, 104.17.164.62
Connecting to aip.scitation.org (aip.scitation.org)|104.17.163.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184842 (181K) [image/jpeg]
Saving to: ‘sample_supplementary_material-1549471999507.jpg’

sample_supplementar 100%[===================>] 180.51K  --.-KB/s    in 0.02s   

2019-11-20 06:58:30 (7.15 MB/s) - ‘sample_supplementary_material-1549471999507.jpg’ saved [184842/184842]



In [2]:
import tensorflow as tf
from keras.layers import Conv2D, Input, BatchNormalization, Dropout, Dense, GlobalAvgPool2D, GlobalMaxPooling2D, MaxPooling2D, Flatten
from keras.optimizers import Adam, RMSprop, Adagrad, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.models import Model, load_model, model_from_json
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications.vgg16 import VGG16
import joblib
import numpy as np
import matplotlib.pyplot as plt
import cv2

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def result(image):

  # We will load our model with their respective weights

  # Holistic Model Loading
  json_file = open('drive/My Drive/Document Image Classification Files/model_holistic.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model_holistic = model_from_json(loaded_model_json)
  # load weights into new model
  model_holistic.load_weights("drive/My Drive/Document Image Classification Files/ content model_holistic.h5")
  print("Loaded model from disk Holistic")
  # Header Region Specific Model Loading

  json_file = open('drive/My Drive/Document Image Classification Files/model_header.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model_header = model_from_json(loaded_model_json)
  # load weights into new model
  model_header.load_weights("drive/My Drive/Document Image Classification Files/ content model_header.h5")
  print("Loaded model from disk Header")

  # Bottom Region Specific Trained Model Loading

  json_file = open('drive/My Drive/Document Image Classification Files/model_bottome.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model_bottom = model_from_json(loaded_model_json)
  # load weights into new model
  model_bottom.load_weights("drive/My Drive/Document Image Classification Files/ content model_bottome.h5")
  print("Loaded model from disk Bottom")

  # Left Region Specific Trained Model Loading

  json_file = open('drive/My Drive/Document Image Classification Files/model_left.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model_left = model_from_json(loaded_model_json)
  # load weights into new model
  model_left.load_weights("drive/My Drive/Document Image Classification Files/ content model_left.h5")
  print("Loaded model from disk Left")

  # Right Region Specific Trained Model Loading

  json_file = open('drive/My Drive/Document Image Classification Files/model_right.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model_right = model_from_json(loaded_model_json)
  # load weights into new model
  model_right.load_weights("drive/My Drive/Document Image Classification Files/ content model_right.h5")
  print("Loaded model from disk Right")

  # Meta Classifier Trained Model Loading

  json_file = open('drive/My Drive/Document Image Classification Files/model_.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model_meta = model_from_json(loaded_model_json)
  # load weights into new model
  model_meta.load_weights("drive/My Drive/Document Image Classification Files/model_metaclassifier.h5")
  print("Loaded model from disk Meta")

  img = plt.imread(image)
  # plt.imshow(img)

  # for whole image
  img_whole = cv2.resize(img,(256,256))
  img_whole = np.reshape(img_whole,[1,256,256,3])/255.
  
  # for header part of an image
  img_header = img[:256,:,:]
  img_header_ = cv2.resize(img_header,(256,256))
  img_header_= np.reshape(img_header_,[1,256,256,3])/255.

  # for bottom part of an image
  img_bottom = img[-256:,:,:]
  img_bottom_ = cv2.resize(img_bottom,(256,256))
  img_bottom_= np.reshape(img_bottom_,[1,256,256,3])/255.

  # for right part of an image
  img_right = img[:,-256:,:]
  img_right_ = cv2.resize(img_right,(256,256))
  img_right_= np.reshape(img_right_,[1,256,256,3])/255.

  # for left part of an image
  img_left = img[:,:256,:]
  img_left_ = cv2.resize(img_left,(256,256))
  img_left_= np.reshape(img_left_,[1,256,256,3])/255.

  # Predicting results on different parts of the image
  features_whole = model_holistic.predict(img_whole)
  features_bottom = model_bottom.predict(img_bottom_)
  features_header = model_bottom.predict(img_header_)
  features_right = model_bottom.predict(img_right_)
  features_left = model_bottom.predict(img_left_)

  # Stacking these predictions
  stacking = np.hstack((features_whole,features_header,features_bottom,features_left,features_right))

  # Meta Classifier
  predictions = model_meta.predict(stacking)
  label = {0:"letter",1:"form",2:"email",3:"handwritten",4:"advertisement",
         5:"scientific report",6:"scientific publication",7:"specification",
         8:"file folder",9:"news article",10:"budget",11:"invoice",12:"presentation",
         13:"questionnaire",14:"resume",15:"memo"}

  result = np.argmax(predictions,axis=1)
  print(result)
  if result[0] in label:
    print(label[result[0]])



  
  

In [10]:
result("/content/sample_supplementary_material-1549471999507.jpg")

Loaded model from disk Holistic
Loaded model from disk Header
Loaded model from disk Bottom
Loaded model from disk Left
Loaded model from disk Right
Loaded model from disk Meta
[15]
memo
